In [1]:
#Winning Factor 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# ML
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

print('Import Done')


Import Done


In [28]:
drivers = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\DRIVERS.csv")
results = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\RESULTS.csv")
races = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\RACES.csv")
constructors = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\CONSTRUCTORS.csv")
qualifying = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\QUALIFYING.csv")
status = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\STATUS.csv")
pit_stops = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\PIT_STOPS.csv")
lap_times = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\LAP_TIMES.csv")

print('Load Done')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 13, saw 2


In [27]:
# Merge relevant tables
merged = results.merge(races, on='raceId', suffixes=('', '_race')) \
                .merge(drivers, on='driverId') \
                .merge(constructors, on='constructorId') \
                .merge(status, on='statusId', how='left') \
                .merge(qualifying[['raceId', 'driverId', 'q1', 'q2', 'q3']], on=['raceId', 'driverId'], how='left')


KeyError: 'raceId'